# 02 • Filtro por Municípios do ER

**Objetivo:** manter somente imóveis dos municípios atendidos pelo ER Jundiaí/SP.

Saída (opcional): `data/processed/imoveis_er_filtrado.csv`


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re, unicodedata

# Caminhos do projeto (ajuste se necessário)
BASE_DIR = Path.cwd().resolve().parents[0]  # .../notebooks -> projeto
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"
OUTPUTS_DIR = BASE_DIR / "outputs"

RAW_DIR, PROCESSED_DIR, OUTPUTS_DIR


(WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/raw'),
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/processed'),
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/outputs'))

In [2]:
def normalize_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.upper()
    s = re.sub(r"\s+", " ", s)
    return s

MUNICIPIOS_ER = ['Jundiaí', 'Cabreúva', 'Caieiras', 'Cajamar', 'Campo Limpo Paulista', 'Francisco Morato', 'Franco da Rocha', 'Itatiba', 'Itupeva', 'Jarinu', 'Louveira', 'Morungaba', 'Pedra Bela', 'Pinhalzinho', 'Tuiutí', 'Vargem', 'Várzea Paulista', 'Bragança Paulista']
MUNICIPIOS_ER_NORM = set(normalize_text(m) for m in MUNICIPIOS_ER)

sorted(MUNICIPIOS_ER)[:5], len(MUNICIPIOS_ER_NORM)


(['Bragança Paulista',
  'Cabreúva',
  'Caieiras',
  'Cajamar',
  'Campo Limpo Paulista'],
 18)

In [3]:
imoveis_csv = RAW_DIR / "Imoveis_SP_01_02_2026.csv"

df = pd.read_csv(imoveis_csv, sep=";", nrows=20000, dtype=str, low_memory=False)
df["MUNICÍPIO_NORM"] = df["MUNICÍPIO"].map(normalize_text)

presentes = set(df["MUNICÍPIO_NORM"].dropna().unique())
er_presentes = sorted(presentes & MUNICIPIOS_ER_NORM)
faltantes = sorted(MUNICIPIOS_ER_NORM - presentes)

len(er_presentes), er_presentes[:10], faltantes[:10]

(18,
 ['BRAGANCA PAULISTA',
  'CABREUVA',
  'CAIEIRAS',
  'CAJAMAR',
  'CAMPO LIMPO PAULISTA',
  'FRANCISCO MORATO',
  'FRANCO DA ROCHA',
  'ITATIBA',
  'ITUPEVA',
  'JARINU'],
 [])

## Filtrar em chunks e gravar CSV (opcional)

In [4]:
out_csv = PROCESSED_DIR / "imoveis_er_filtrado.csv"
out_csv.parent.mkdir(parents=True, exist_ok=True)

first = True
kept = 0

for chunk in pd.read_csv(imoveis_csv, sep=";", dtype=str, chunksize=200_000, low_memory=False):
    chunk["MUNICÍPIO_NORM"] = chunk["MUNICÍPIO"].map(normalize_text)
    chunk = chunk[chunk["MUNICÍPIO_NORM"].isin(MUNICIPIOS_ER_NORM)]
    kept += len(chunk)

    if len(chunk) == 0:
        continue

    chunk.to_csv(
        out_csv,
        sep=";",
        index=False,
        mode=("w" if first else "a"),
        header=first,
        encoding="utf-8"
    )
    first = False

kept, out_csv

(26332,
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/processed/imoveis_er_filtrado.csv'))

In [5]:
df_f = pd.read_csv(out_csv, sep=";", nrows=5000, dtype=str)
df_f["MUNICÍPIO"].value_counts().head(20)

MUNICÍPIO
BRAGANÇA PAULISTA       980
CABREÚVA                820
JUNDIAÍ                 666
PINHALZINHO             418
ITATIBA                 400
JARINU                  296
ITUPEVA                 267
PEDRA BELA              195
CAMPO LIMPO PAULISTA    156
LOUVEIRA                154
MORUNGABA               152
VARGEM                  137
TUIUTI                  131
FRANCO DA ROCHA          86
CAJAMAR                  65
VÁRZEA PAULISTA          47
CAIEIRAS                 17
FRANCISCO MORATO         13
Name: count, dtype: int64